In [5]:
import numpy as np
import pandas as pd
import time
import tkinter as tk
from openpyxl import load_workbook
import undetected_chromedriver as uc
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from tabulate import tabulate
from webdriver_manager.chrome import ChromeDriverManager
servico = Service(ChromeDriverManager().install())
import funcoes as fc

In [6]:
#INICIA O NAVEGADOR
navegador = uc.Chrome(service=servico)
navegador.maximize_window()
navegador.get('https://sei.anatel.gov.br/')


In [ ]:
lista_procConformes = ["53500.074124/2024-32", "53500.073842/2024-91"]

In [7]:
#FUNCAO QUE ESPERA UM ELEMENTO CARREGAR NA TELA E CLICA NELE
def clica_noelemento(navegador, modo_procura, element_id):
    # Espera até que o elemento seja carregado (exemplo: elemento localizado por ID)
    try:
        element = WebDriverWait(navegador, 10).until(
            EC.element_to_be_clickable((modo_procura, element_id))
        )
        # Clica no elemento
        element.click()
    except TimeoutException:
        print(f"Elemento {element_id} não foi carregado no tempo esperado")


In [9]:
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrVisualizacao')
time.sleep(1)
#CLICA NO ICONE DE LEGO
# clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[8]')
#navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[8]').click()
#SELECIONA BLOCO (SELECIONA O PRIMEIRO DESPACHO PARA DRONES APROVADOS QUE LER)
time.sleep(1.5)
select_element = navegador.find_element(By.ID, 'selBloco')
time.sleep(0.5)
select = Select(select_element)
time.sleep(0.5)
#PROCURA O PRIMEIRO BLOCO QUE TENHA O TEXTO "Despachos para Drones aprovados"
for opcao in select.options:
    if "Despachos para Drones aprovados" in opcao.text:
        select.select_by_visible_text(opcao.text)
        break
#CLICA NO BOTAO DE INCLUIR NO BLOCO